# 4. Multi-Dimensional Steering

This notebook extends the contrastive steering approach to **multiple demographic dimensions**:

1. **Party:** Republican vs Democrat
2. **Race:** White vs Black
3. **Education:** College vs No College

## Research Questions

1. Do different demographic dimensions capture independent variation?
2. Can we simulate specific demographic profiles (e.g., "college-educated Black Democrat")?
3. How do multi-dimensional steered responses compare to GSS subgroup data?

## Approach

Multi-dimensional steering vector:
```
v_multi = α(v_Rep - v_Dem) + β(v_White - v_Black) + γ(v_College - v_NoCollege)
```

Where α, β, γ are steering strengths for each dimension.

## 1. Setup & Imports

In [ ]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

import os
BASE_DIR = "/content/drive/MyDrive/belief_and_llms"
os.makedirs(BASE_DIR, exist_ok=True)
print(f"Base directory: {BASE_DIR}")

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import torch.nn.functional as F

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded: {model_name}")
print(f"Number of layers: {len(model.model.layers)}")

## 3. Load Demographic Vectors

In [ ]:
# Load vectors from the best layer (Layer 8 from previous analysis)
STEERING_LAYER = 8

vectors_path = os.path.join(BASE_DIR, f"vectors/gss_demographic_vectors_layer{STEERING_LAYER}.pt")
vectors_data = torch.load(vectors_path)

print(f"Loaded vectors from layer {STEERING_LAYER}")
print(f"Available demographics: {len(vectors_data['vectors'])}")

# List all available demographic keys
for key in sorted(vectors_data['vectors'].keys()):
    print(f"  - {key}")

## 4. Create Multi-Dimensional Contrastive Vectors

In [ ]:
# Define the contrastive pairs for each dimension
# Note: Adjust these keys based on actual vector names in your data

CONTRASTIVE_DIMENSIONS = {
    "party": {
        "positive": "PartyID_Strong Republican",  # Positive direction
        "negative": "PartyID_Strong Democrat",    # Negative direction
        "label": "Republican ↔ Democrat"
    },
    "race": {
        "positive": "Race_White person",
        "negative": "Race_Black person",
        "label": "White ↔ Black"
    },
    "education": {
        "positive": "Degree_person with a graduate degree",  # or college degree
        "negative": "Degree_person with a high school diploma",
        "label": "College ↔ No College"
    }
}

# Create contrastive vectors for each dimension
contrastive_vectors = {}

for dim_name, dim_config in CONTRASTIVE_DIMENSIONS.items():
    pos_key = dim_config["positive"]
    neg_key = dim_config["negative"]
    
    if pos_key in vectors_data['vectors'] and neg_key in vectors_data['vectors']:
        pos_vec = vectors_data['vectors'][pos_key]
        neg_vec = vectors_data['vectors'][neg_key]
        contrastive_vec = pos_vec - neg_vec
        
        contrastive_vectors[dim_name] = {
            "vector": contrastive_vec,
            "label": dim_config["label"],
            "positive": pos_key,
            "negative": neg_key
        }
        print(f"{dim_name}: {dim_config['label']}")
        print(f"  Vector norm: {contrastive_vec.norm().item():.4f}")
    else:
        print(f"WARNING: Could not find vectors for {dim_name}")
        print(f"  Missing: {pos_key if pos_key not in vectors_data['vectors'] else neg_key}")

In [ ]:
# Check correlation between contrastive dimensions
# If highly correlated, they capture similar variation

print("=== Correlation Between Contrastive Dimensions ===")
print("(Low correlation = independent dimensions)\n")

dim_names = list(contrastive_vectors.keys())
correlation_matrix = np.zeros((len(dim_names), len(dim_names)))

for i, dim1 in enumerate(dim_names):
    for j, dim2 in enumerate(dim_names):
        v1 = contrastive_vectors[dim1]["vector"]
        v2 = contrastive_vectors[dim2]["vector"]
        
        # Cosine similarity
        cos_sim = F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0)).item()
        correlation_matrix[i, j] = cos_sim

# Display as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt=".3f", 
            xticklabels=[contrastive_vectors[d]["label"] for d in dim_names],
            yticklabels=[contrastive_vectors[d]["label"] for d in dim_names],
            cmap="RdBu_r", center=0, vmin=-1, vmax=1)
plt.title("Cosine Similarity Between Contrastive Dimensions")
plt.tight_layout()
plt.show()

# Print correlation values
for i, dim1 in enumerate(dim_names):
    for j, dim2 in enumerate(dim_names):
        if i < j:
            print(f"{dim1} ↔ {dim2}: {correlation_matrix[i,j]:.3f}")

## 5. Define Multi-Dimensional Steering Function

In [ ]:
def get_multi_dim_steering_vector(dimension_strengths):
    """
    Create a combined steering vector from multiple dimensions.
    
    Args:
        dimension_strengths: dict mapping dimension name to strength
            e.g., {"party": 2.0, "race": 1.0, "education": -1.0}
    
    Returns:
        Combined steering vector
    """
    combined_vector = None
    
    for dim_name, strength in dimension_strengths.items():
        if dim_name in contrastive_vectors:
            vec = contrastive_vectors[dim_name]["vector"] * strength
            if combined_vector is None:
                combined_vector = vec
            else:
                combined_vector = combined_vector + vec
    
    return combined_vector


def get_binary_choice_prob_multi(prompt, dimension_strengths, layer_idx):
    """
    Get probability of choosing option A vs B with multi-dimensional steering.
    
    Args:
        prompt: Binary choice prompt
        dimension_strengths: dict mapping dimension name to strength
        layer_idx: Layer to inject at
    
    Returns:
        Probability of option A
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Get combined steering vector
    steering_vector = get_multi_dim_steering_vector(dimension_strengths)
    
    if steering_vector is None:
        # No steering, just run model
        with torch.no_grad():
            outputs = model(**inputs)
    else:
        steer_vec = steering_vector.to(model.device).to(model.dtype)
        
        def steering_hook(module, input, output):
            perturbation = steer_vec.view(1, 1, -1)
            if isinstance(output, tuple):
                modified_hidden = output[0] + perturbation
                return (modified_hidden,) + output[1:]
            elif isinstance(output, torch.Tensor):
                return output + perturbation
            return output
        
        layer = model.model.layers[layer_idx]
        handle = layer.register_forward_hook(steering_hook)
        
        try:
            with torch.no_grad():
                outputs = model(**inputs)
        finally:
            handle.remove()
    
    # Get logits for next token
    next_token_logits = outputs.logits[0, -1, :]
    
    # Get token IDs for A and B
    id_a = tokenizer.encode(" A", add_special_tokens=False)[-1]
    id_b = tokenizer.encode(" B", add_special_tokens=False)[-1]
    
    # Compute probabilities
    probs = F.softmax(torch.tensor([next_token_logits[id_a], next_token_logits[id_b]]), dim=0)
    
    return probs[0].item()

print("Multi-dimensional steering functions defined.")

## 6. Define Policy Questions & GSS Mappings

In [ ]:
# Policy questions (same as before)
POLICY_QUESTIONS = {
    "abortion": {
        "binary_prompt": "[INST] What is your stance on abortion? [/INST]\n(A) Pro-choice (woman should have the right to choose)\n(B) Pro-life (abortion should be restricted or banned)\nAnswer:",
        "option_a_label": "Pro-choice",
        "option_b_label": "Pro-life",
        "gss_variable": "ABANY"
    },
    "death_penalty": {
        "binary_prompt": "[INST] What is your stance on the death penalty for persons convicted of murder? [/INST]\n(A) Oppose the death penalty\n(B) Favor the death penalty\nAnswer:",
        "option_a_label": "Oppose",
        "option_b_label": "Favor",
        "gss_variable": "CAPPUN"
    },
    "gun_control": {
        "binary_prompt": "[INST] What is your stance on gun control? [/INST]\n(A) Support stricter gun control laws\n(B) Protect Second Amendment gun rights\nAnswer:",
        "option_a_label": "Stricter control",
        "option_b_label": "Protect 2A",
        "gss_variable": "GUNLAW"
    }
}

print(f"Defined {len(POLICY_QUESTIONS)} policy questions.")

## 7. Load GSS Data for Subgroup Analysis

In [ ]:
# Load GSS data
gss_path = os.path.join(BASE_DIR, "data/gss_cleaned.csv")
gss_df = pd.read_csv(gss_path)

print(f"Loaded GSS data: {len(gss_df)} rows")
print(f"Columns: {list(gss_df.columns)}")
print(f"\nYear range: {gss_df['year'].min()} - {gss_df['year'].max()}")

In [ ]:
# Create binary variables for multi-dimensional filtering

# Party: Democrat (0,1,2) vs Republican (4,5,6)
gss_df['party_binary'] = gss_df['PARTYID'].apply(
    lambda x: 'Democrat' if x in [0, 1, 2] else ('Republican' if x in [4, 5, 6] else 'Independent')
)

# Race: White vs Black (from 'race' column)
# Note: Check actual values in your data
print("Race values:", gss_df['race'].unique())

# Education: College vs No College (from 'degree' column)
print("Degree values:", gss_df['degree'].unique())

In [ ]:
# Create education binary
# Adjust based on actual values in your data
college_degrees = ['Bachelor', 'Graduate']  # Adjust these
gss_df['education_binary'] = gss_df['degree'].apply(
    lambda x: 'College' if x in college_degrees else 'No College'
)

# Create race binary (White vs Black only for this analysis)
gss_df['race_binary'] = gss_df['race'].apply(
    lambda x: x if x in ['White', 'Black'] else 'Other'
)

print("Party distribution:")
print(gss_df['party_binary'].value_counts())
print("\nEducation distribution:")
print(gss_df['education_binary'].value_counts())
print("\nRace distribution:")
print(gss_df['race_binary'].value_counts())

In [ ]:
def compute_gss_subgroup_prob(gss_df, topic, party=None, race=None, education=None):
    """
    Compute GSS probability for a demographic subgroup.
    
    Args:
        gss_df: GSS dataframe
        topic: Policy topic
        party: 'Democrat', 'Republican', or None
        race: 'White', 'Black', or None
        education: 'College', 'No College', or None
    
    Returns:
        Probability of Option A, sample size
    """
    config = POLICY_QUESTIONS[topic]
    var = config["gss_variable"]
    
    if var not in gss_df.columns:
        return None, 0
    
    # Filter by demographics
    filtered_df = gss_df.copy()
    
    if party:
        filtered_df = filtered_df[filtered_df['party_binary'] == party]
    if race:
        filtered_df = filtered_df[filtered_df['race_binary'] == race]
    if education:
        filtered_df = filtered_df[filtered_df['education_binary'] == education]
    
    # Define option A values based on topic
    if topic == "abortion":
        option_a_values = [1]  # YES
        option_b_values = [2]  # NO
    elif topic == "death_penalty":
        option_a_values = [2]  # OPPOSE
        option_b_values = [1]  # FAVOR
    elif topic == "gun_control":
        option_a_values = [1]  # FAVOR
        option_b_values = [2]  # OPPOSE
    else:
        return None, 0
    
    valid_values = option_a_values + option_b_values
    valid_df = filtered_df[filtered_df[var].isin(valid_values)]
    
    if len(valid_df) == 0:
        return None, 0
    
    option_a_count = valid_df[var].isin(option_a_values).sum()
    prob_a = option_a_count / len(valid_df)
    
    return prob_a, len(valid_df)

print("GSS subgroup function defined.")

## 8. Test Single Dimension Effects

In [ ]:
# First, test each dimension independently
print("=== SINGLE DIMENSION STEERING EFFECTS ===")
print(f"Layer: {STEERING_LAYER}\n")

single_dim_results = []

for topic in POLICY_QUESTIONS.keys():
    prompt = POLICY_QUESTIONS[topic]["binary_prompt"]
    print(f"\n{topic.upper()}:")
    
    # Baseline (no steering)
    baseline_prob = get_binary_choice_prob_multi(prompt, {}, STEERING_LAYER)
    print(f"  Baseline: P(A) = {baseline_prob:.1%}")
    
    for dim_name in contrastive_vectors.keys():
        # Test positive and negative steering
        for strength in [-2.0, 2.0]:
            prob = get_binary_choice_prob_multi(prompt, {dim_name: strength}, STEERING_LAYER)
            direction = "positive" if strength > 0 else "negative"
            
            single_dim_results.append({
                "topic": topic,
                "dimension": dim_name,
                "strength": strength,
                "prob_a": prob,
                "delta": prob - baseline_prob
            })
            
            print(f"  {dim_name} ({strength:+.1f}): P(A) = {prob:.1%} (Δ = {prob - baseline_prob:+.1%})")

single_dim_df = pd.DataFrame(single_dim_results)
print("\nDone!")

## 9. Test Multi-Dimensional Combinations

In [ ]:
# Define demographic profiles to simulate
# Positive strength = toward White/Republican/College
# Negative strength = toward Black/Democrat/NoCollege

DEMOGRAPHIC_PROFILES = {
    "White Republican College": {"party": 2.0, "race": 2.0, "education": 2.0},
    "White Republican No-College": {"party": 2.0, "race": 2.0, "education": -2.0},
    "White Democrat College": {"party": -2.0, "race": 2.0, "education": 2.0},
    "White Democrat No-College": {"party": -2.0, "race": 2.0, "education": -2.0},
    "Black Republican College": {"party": 2.0, "race": -2.0, "education": 2.0},
    "Black Republican No-College": {"party": 2.0, "race": -2.0, "education": -2.0},
    "Black Democrat College": {"party": -2.0, "race": -2.0, "education": 2.0},
    "Black Democrat No-College": {"party": -2.0, "race": -2.0, "education": -2.0},
}

print(f"Defined {len(DEMOGRAPHIC_PROFILES)} demographic profiles to test.")

In [ ]:
# Test all demographic profiles
print("=== MULTI-DIMENSIONAL STEERING RESULTS ===")
print(f"Layer: {STEERING_LAYER}\n")

multi_dim_results = []

for topic in POLICY_QUESTIONS.keys():
    prompt = POLICY_QUESTIONS[topic]["binary_prompt"]
    print(f"\n{topic.upper()}:")
    
    for profile_name, strengths in DEMOGRAPHIC_PROFILES.items():
        prob = get_binary_choice_prob_multi(prompt, strengths, STEERING_LAYER)
        
        multi_dim_results.append({
            "topic": topic,
            "profile": profile_name,
            "party_strength": strengths.get("party", 0),
            "race_strength": strengths.get("race", 0),
            "education_strength": strengths.get("education", 0),
            "llm_prob_a": prob
        })
        
        print(f"  {profile_name}: P(A) = {prob:.1%}")

multi_dim_df = pd.DataFrame(multi_dim_results)
print("\nDone!")

## 10. Compare with GSS Subgroups

In [ ]:
# Compute GSS probabilities for matching subgroups
print("=== GSS SUBGROUP PROBABILITIES ===")

gss_subgroup_results = []

for topic in POLICY_QUESTIONS.keys():
    print(f"\n{topic.upper()}:")
    
    for party in ['Democrat', 'Republican']:
        for race in ['White', 'Black']:
            for education in ['College', 'No College']:
                prob, n = compute_gss_subgroup_prob(
                    gss_df, topic, 
                    party=party, race=race, education=education
                )
                
                profile_name = f"{race} {party} {education.replace(' ', '-')}"
                
                gss_subgroup_results.append({
                    "topic": topic,
                    "profile": profile_name,
                    "party": party,
                    "race": race,
                    "education": education,
                    "gss_prob_a": prob,
                    "gss_n": n
                })
                
                if prob is not None:
                    print(f"  {profile_name}: P(A) = {prob:.1%} (n={n})")
                else:
                    print(f"  {profile_name}: No data")

gss_subgroup_df = pd.DataFrame(gss_subgroup_results)
print("\nDone!")

In [ ]:
# Merge LLM and GSS results for comparison
comparison_df = multi_dim_df.merge(
    gss_subgroup_df[["topic", "profile", "gss_prob_a", "gss_n"]], 
    on=["topic", "profile"], 
    how="left"
)

# Calculate error
comparison_df["error"] = abs(comparison_df["llm_prob_a"] - comparison_df["gss_prob_a"])

print("=== LLM vs GSS COMPARISON ===")
print(comparison_df.to_string(index=False))

print(f"\nMean absolute error: {comparison_df['error'].mean():.3f}")

## 11. Visualization

In [ ]:
# Visualize LLM vs GSS for each topic
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, topic in enumerate(POLICY_QUESTIONS.keys()):
    ax = axes[idx]
    topic_df = comparison_df[comparison_df["topic"] == topic].copy()
    
    # Sort by GSS probability for better visualization
    topic_df = topic_df.sort_values("gss_prob_a")
    
    x = range(len(topic_df))
    width = 0.35
    
    bars1 = ax.bar([i - width/2 for i in x], topic_df["gss_prob_a"], width, 
                   label="GSS", color="steelblue", alpha=0.8)
    bars2 = ax.bar([i + width/2 for i in x], topic_df["llm_prob_a"], width, 
                   label="LLM", color="coral", alpha=0.8)
    
    ax.set_xlabel("Demographic Profile")
    ax.set_ylabel("P(Option A)")
    ax.set_title(f"{topic.replace('_', ' ').title()}")
    ax.set_xticks(x)
    ax.set_xticklabels(topic_df["profile"], rotation=45, ha="right", fontsize=8)
    ax.legend()
    ax.set_ylim(0, 1)
    ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)

plt.suptitle("Multi-Dimensional Steering: LLM vs GSS Subgroups", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: LLM vs GSS probabilities
plt.figure(figsize=(10, 8))

colors = {'abortion': 'red', 'death_penalty': 'blue', 'gun_control': 'green'}

for topic in POLICY_QUESTIONS.keys():
    topic_df = comparison_df[comparison_df["topic"] == topic]
    plt.scatter(topic_df["gss_prob_a"], topic_df["llm_prob_a"], 
                c=colors[topic], label=topic.replace('_', ' ').title(), 
                s=100, alpha=0.7)

# Perfect correlation line
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect match')

plt.xlabel("GSS Probability (Ground Truth)")
plt.ylabel("LLM Probability (Steered)")
plt.title("Multi-Dimensional Steering Accuracy")
plt.legend()
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate correlation
valid_df = comparison_df.dropna(subset=["gss_prob_a", "llm_prob_a"])
correlation = valid_df["gss_prob_a"].corr(valid_df["llm_prob_a"])
print(f"Correlation between LLM and GSS: {correlation:.3f}")

## 12. Save Results

In [ ]:
# Save all results
single_dim_df.to_csv(os.path.join(BASE_DIR, f"multi_dim_single_effects_layer{STEERING_LAYER}.csv"), index=False)
multi_dim_df.to_csv(os.path.join(BASE_DIR, f"multi_dim_steering_results_layer{STEERING_LAYER}.csv"), index=False)
gss_subgroup_df.to_csv(os.path.join(BASE_DIR, f"gss_subgroup_probabilities.csv"), index=False)
comparison_df.to_csv(os.path.join(BASE_DIR, f"multi_dim_llm_vs_gss_comparison_layer{STEERING_LAYER}.csv"), index=False)

print("Results saved:")
print(f"  - multi_dim_single_effects_layer{STEERING_LAYER}.csv")
print(f"  - multi_dim_steering_results_layer{STEERING_LAYER}.csv")
print(f"  - gss_subgroup_probabilities.csv")
print(f"  - multi_dim_llm_vs_gss_comparison_layer{STEERING_LAYER}.csv")

## 13. Summary & Analysis

In [ ]:
# Summary statistics
print("=== MULTI-DIMENSIONAL STEERING SUMMARY ===")
print(f"\nLayer: {STEERING_LAYER}")
print(f"Demographic dimensions: Party, Race, Education")
print(f"Number of profiles tested: {len(DEMOGRAPHIC_PROFILES)}")
print(f"Policy topics: {list(POLICY_QUESTIONS.keys())}")

# Dimension independence
print("\n--- Dimension Correlations ---")
for i, dim1 in enumerate(dim_names):
    for j, dim2 in enumerate(dim_names):
        if i < j:
            print(f"{dim1} ↔ {dim2}: {correlation_matrix[i,j]:.3f}")

# Accuracy by topic
print("\n--- Mean Absolute Error by Topic ---")
for topic in POLICY_QUESTIONS.keys():
    topic_error = comparison_df[comparison_df["topic"] == topic]["error"].mean()
    print(f"{topic}: {topic_error:.3f}")

print(f"\nOverall MAE: {comparison_df['error'].mean():.3f}")
print(f"Overall correlation: {correlation:.3f}")